# Example 1 - OpenWeatherMap API — Real Sensor-like Weather Data

In [2]:
import requests, time
from river import linear_model, optim, preprocessing, metrics

model = preprocessing.StandardScaler() | linear_model.LinearRegression(
    optimizer=optim.SGD(0.01)
)
mae = metrics.MAE()

URL = "https://api.open-meteo.com/v1/forecast?latitude=48.85&longitude=2.35&current_weather=true"

try:
    while True:
        data = requests.get(URL).json()["current_weather"]
        temp = data["temperature"]
        wind = data["windspeed"]

        x = {"wind": wind}
        y = temp

        y_pred = model.predict_one(x)
        model.learn_one(x, y)
        mae.update(y, y_pred)

        print(f"Temp={y:.2f}°C | Wind={wind:.1f} km/h | Pred={y_pred:.2f}°C | MAE={mae.get():.3f}")
        time.sleep(2)  # 5 min delay
except KeyboardInterrupt:
    print("\nStopped streaming.")


Temp=17.00°C | Wind=7.4 km/h | Pred=0.00°C | MAE=17.000
Temp=17.00°C | Wind=7.4 km/h | Pred=0.34°C | MAE=16.830
Temp=17.00°C | Wind=7.4 km/h | Pred=0.67°C | MAE=16.662
Temp=17.00°C | Wind=7.4 km/h | Pred=1.00°C | MAE=16.497
Temp=17.00°C | Wind=7.4 km/h | Pred=1.32°C | MAE=16.333

Stopped streaming.


# Linear Regressuib (SGD) with ADWIN Drift Detector

In [ ]:
import requests, time
from river import linear_model, optim, preprocessing, metrics, drift

# --- Model Setup ---
model = preprocessing.StandardScaler() | linear_model.LinearRegression(
    optimizer=optim.SGD(0.01)
)
mae = metrics.MAE()

# --- Drift Detector ---
adwin = drift.ADWIN(delta=0.002)

URL = "https://api.open-meteo.com/v1/forecast?latitude=48.85&longitude=2.35&current_weather=true"

print("Starting online regression with concept drift detection. Press Ctrl+C to stop.\n")

try:
    while True:
        # Fetch current weather data
        response = requests.get(URL)
        data = response.json()["current_weather"]
        temp = data["temperature"]
        wind = data["windspeed"]

        # Define input/output
        x = {"wind": wind}
        y = temp

        # Predict and learn
        y_pred = model.predict_one(x)
        model.learn_one(x, y)

        # Compute error and update detector
        error = abs(y - (y_pred or 0))
        adwin.update(error)
        mae.update(y, y_pred)

        # Check for drift (new API property name)
        drift_flag = "⚠️ Drift Detected!" if adwin.drift_detected else ""
        print(
            f"Temp={y:.2f}°C | Wind={wind:.1f} km/h | Pred={y_pred:.2f}°C "
            f"| MAE={mae.get():.3f} | Error={error:.3f} {drift_flag}"
        )

        # Reset model if drift detected
        if adwin.drift_detected:
            print("🔄 Resetting model due to detected drift...\n")
            model = preprocessing.StandardScaler() | linear_model.LinearRegression(
                optimizer=optim.SGD(0.01)
            )
            adwin = drift.ADWIN(delta=0.002)

        time.sleep(300)  # 5 min delay (API refresh)

except KeyboardInterrupt:
    print("\nStopped streaming.")


🌦️ Starting online regression with concept drift detection. Press Ctrl+C to stop.

Temp=16.50°C | Wind=9.4 km/h | Pred=0.00°C | MAE=16.500 | Error=16.500 

Stopped streaming.


# River + Evidently integration

In [3]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning, module="numpy")

import time
import datetime
import requests
import pandas as pd
from river import linear_model, optim, preprocessing, metrics, drift

# --- Evidently imports ---
from evidently import ColumnMapping
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset, DataQualityPreset
from evidently.ui.workspace.cloud import CloudWorkspace 

#Cloud workspace setup ---
EVIDENTLY_TOKEN = "dG9rbgFHWHjVjJJGn7OGiLb7In9PH5OJ7tY5R62E5489OOcAowBA1Yh1S64SAheDkX1A+G8SyXd6boBWdoxemLs1v7+19jEkVAYn2/KoTxzigAR/lHqDF4qC7fKhv1CZENDCVG8agSeP4ts="
EVIDENTLY_URL = "https://app.evidently.cloud"
EVIDENTLY_PROJECT_ID = "0199b983-1622-7224-a32f-2f608ab8d49e"

ws = CloudWorkspace(token=EVIDENTLY_TOKEN, url="https://app.evidently.cloud/v2")
project = ws.get_project(EVIDENTLY_PROJECT_ID)

print(f"Connected to Evidently Cloud project: {project.id}")

# --- Data schema ---
column_mapping = ColumnMapping()
column_mapping.target = "temperature"
column_mapping.prediction = "prediction"
column_mapping.numerical_features = ["wind"]

# --- River model ---
model = preprocessing.StandardScaler() | linear_model.LinearRegression(optimizer=optim.SGD(0.01))
mae = metrics.MAE()
adwin = drift.ADWIN()

reference_window, current_window = [], []

# --- Example API ---
URL = "https://api.open-meteo.com/v1/forecast?latitude=48.85&longitude=2.35&current_weather=true"

def create_data_report(ref_df, cur_df, ts):
    report = Report(
        metrics=[
            DataQualityPreset(),
            DataDriftPreset(stattest="psi", stattest_threshold=0.3),
        ],
        timestamp=ts,
    )
    report.run(reference_data=ref_df, current_data=cur_df)
    return report

try:
    while True:
        data = requests.get(URL).json()["current_weather"]
        temp = data["temperature"]
        wind = data["windspeed"]

        x = {"wind": wind}
        y_pred = model.predict_one(x)
        model.learn_one(x, temp)
        mae.update(temp, y_pred)
        adwin.update(abs(temp - (y_pred or 0)))

        print(f"Temp={temp:.2f}°C | Pred={y_pred:.2f}°C | MAE={mae.get():.3f} | Drift={adwin.drift_detected}")

        current_window.append({"temperature": temp, "wind": wind, "prediction": y_pred})

        if len(current_window) >= 2:
            if not reference_window:
                reference_window = current_window.copy()
                current_window = []
                print("📘 Initialized reference window.")
            else:
                ref_df = pd.DataFrame(reference_window)
                cur_df = pd.DataFrame(current_window)
                ts = datetime.datetime.now()

                # --- Create report ---
                report = create_data_report(ref_df, cur_df, ts)

                # --- Correct way to send to Cloud --

                ws.add_report(project.id, report)
                print(f"Uploaded report snapshot ({ts.isoformat()})")

                reference_window = current_window.copy()
                current_window = []

        time.sleep(2)

except KeyboardInterrupt:
    print("\nStreaming stopped.")


Connected to Evidently Cloud project: 0199b983-1622-7224-a32f-2f608ab8d49e
Temp=16.60°C | Pred=0.00°C | MAE=16.600 | Drift=False
Temp=16.60°C | Pred=0.33°C | MAE=16.434 | Drift=False
📘 Initialized reference window.

Streaming stopped.


In [2]:
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset
import pandas as pd
import datetime

dummy = pd.DataFrame({"a": [1,2,3,4,5], "b": [2,3,4,5,6]})
report = Report(metrics=[DataDriftPreset()], timestamp=datetime.datetime.now())
report.run(reference_data=dummy, current_data=dummy)

print("Trying upload...")
ws.add_report(project.id, report)
print("✅ Upload successful")


Trying upload...
✅ Upload successful


# Local Version

In [ ]:
import os
import time
import datetime
import warnings
import requests
import pandas as pd
import numpy as np
from river import linear_model, optim, preprocessing, metrics, drift
from evidently import ColumnMapping
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset, DataQualityPreset

# --- ⚙️ Suppress irrelevant warnings ---
warnings.filterwarnings("ignore", category=RuntimeWarning, module="numpy")

# --- Column mapping for reports ---
column_mapping = ColumnMapping()
column_mapping.target = "temperature"
column_mapping.prediction = "prediction"
column_mapping.numerical_features = ["wind"]

# --- River model setup ---
model = preprocessing.StandardScaler() | linear_model.LinearRegression(optimizer=optim.SGD(0.01))
mae = metrics.MAE()
adwin = drift.ADWIN()

# --- Buffers for drift analysis ---
reference_window, current_window = [], []

# --- Example API (weather) ---
URL = "https://api.open-meteo.com/v1/forecast?latitude=48.85&longitude=2.35&current_weather=true"

# --- Create output folder ---
os.makedirs("reports", exist_ok=True)

def create_data_report(ref_df, cur_df, ts):
    """Create Evidently Data Drift + Quality report for one batch."""
    report = Report(
        metrics=[
            DataQualityPreset(),
            DataDriftPreset(stattest="psi", stattest_threshold=0.3),
        ],
        timestamp=ts,
    )
    report.run(reference_data=ref_df, current_data=cur_df)
    return report

try:
    while True:
        # --- 1️⃣ Fetch live data ---
        data = requests.get(URL).json()["current_weather"]
        temp = data["temperature"]
        wind = data["windspeed"]

        # --- 2️⃣ Predict and learn online ---
        x = {"wind": wind}
        y_pred = model.predict_one(x)
        model.learn_one(x, temp)
        mae.update(temp, y_pred)
        adwin.update(abs(temp - (y_pred or 0)))

        drift_flag = adwin.drift_detected
        print(f"Temp={temp:.2f}°C | Pred={y_pred:.2f}°C | MAE={mae.get():.3f} | Drift={drift_flag}")

        # --- Add to rolling window ---
        current_window.append({"temperature": temp, "wind": wind, "prediction": y_pred})

        # --- Every 20 samples, save a new local report ---
        if len(current_window) >= 5:
            if not reference_window:
                reference_window = current_window.copy()
                current_window = []
                print("📘 Initialized reference window.")
            else:
                ref_df = pd.DataFrame(reference_window)
                cur_df = pd.DataFrame(current_window)

                # Add small noise to avoid NaN drift
                cur_df["wind"] += np.random.randn(len(cur_df)) * 0.1

                ts = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

                report = create_data_report(ref_df, cur_df, datetime.datetime.now())

                # --- Save locally as HTML ---
                file_path = f"reports/weather_drift_report_{ts}.html"
                report.save_html(file_path)
                print(f" Saved report: {file_path}")

                reference_window = current_window.copy()
                current_window = []

        time.sleep(2)

except KeyboardInterrupt:
    print("\n Streaming stopped.")


🌡️ Temp=16.60°C | Pred=0.00°C | MAE=16.600 | Drift=False
🌡️ Temp=16.60°C | Pred=0.33°C | MAE=16.434 | Drift=False
🌡️ Temp=16.60°C | Pred=0.66°C | MAE=16.270 | Drift=False
🌡️ Temp=16.60°C | Pred=0.98°C | MAE=16.109 | Drift=False
🌡️ Temp=16.60°C | Pred=1.29°C | MAE=15.949 | Drift=False
📘 Initialized reference window.
🌡️ Temp=16.60°C | Pred=1.59°C | MAE=15.792 | Drift=False
🌡️ Temp=16.60°C | Pred=1.90°C | MAE=15.637 | Drift=False
🌡️ Temp=16.60°C | Pred=2.19°C | MAE=15.483 | Drift=False
🌡️ Temp=16.60°C | Pred=2.48°C | MAE=15.332 | Drift=False
🌡️ Temp=16.60°C | Pred=2.76°C | MAE=15.183 | Drift=False
 Saved report: reports/weather_drift_report_20251008_174606.html
🌡️ Temp=16.60°C | Pred=3.04°C | MAE=15.036 | Drift=False
🌡️ Temp=16.60°C | Pred=3.31°C | MAE=14.890 | Drift=False
🌡️ Temp=16.60°C | Pred=3.57°C | MAE=14.747 | Drift=False
🌡️ Temp=16.60°C | Pred=3.83°C | MAE=14.606 | Drift=False
🌡️ Temp=16.60°C | Pred=4.09°C | MAE=14.466 | Drift=False
 Saved report: reports/weather_drift_report_2025